In [1]:
import boto3
import json
import os.path
session = boto3.Session(profile_name='later-sandbox')

In [2]:
BUCKET = "faces-cnn"

In [3]:
s3 = session.resource('s3')

bucket = s3.Bucket(BUCKET)

In [4]:
def detect_faces(bucket, key, region="us-east-1"):
	rekognition = session.client("rekognition", region)
	response = rekognition.detect_faces(
		Image={
			"S3Object": {
				"Bucket": bucket,
				"Name": key,
			}
		},
		Attributes=[
            'ALL',
        ]
	)
	return response['FaceDetails']

In [68]:
idx = 0

def process_obj(obj):
    global idx
    idx = idx+1
    key = obj.key
    filename = key.split('/')[1]
    filename = filename.split('.')[0]
    fname = "data/flw-responses/{}.json".format(filename)
    
    if os.path.isfile(fname):
        print(key, " Processed")
    else: 
        print(idx, ' - ', key)
        for response in detect_faces(BUCKET, key):
            filename = key.split('/')[1]
            filename = filename.split('.')[0]
            key = "data/flw-responses/{}.json".format(filename)
            with open(key, 'w') as outfile:
                json.dump(response, outfile)

In [70]:
my_array= bucket.objects.filter(Prefix='flw-raw')
    
from multiprocessing.dummy import Pool as ThreadPool 
pool = ThreadPool(4) 
results = pool.map(process_obj, my_array)

KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2
import time 

def detect_faces(f_cascade, colored_img, scaleFactor = 1.1):
    img_copy = np.copy(colored_img)
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
    
    #let's detect multiscale (some images may be closer to camera than others) images
    faces = f_cascade.detectMultiScale(gray, scaleFactor=scaleFactor, minNeighbors=5);
    
    #go over list of faces and draw them as rectangles on original colored img
    if len(faces) == 1:
        (x, y, w, h) = faces[0]
        return img_copy[y:y+h, x:x+w]
    else:
        return []

In [13]:
import glob
import json
import os
import cv2
idx = 0

haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

for filename in glob.glob('data/flw-raw/*.jpg'):
    idx = idx + 1
    test_img = cv2.imread(filename)
    new_filename = filename.replace('flw-raw', 'flw-resize')
    
    faces_detected_img = detect_faces(haar_face_cascade, test_img)
    
    if len(faces_detected_img) > 0:
        
        resize_detected = cv2.resize(faces_detected_img, (224,224))
        
        cv2.imwrite(new_filename, resize_detected)
    #cv2.imshow('Test Imag', resize_detected)
    #cv2.waitKey(0)

    

In [18]:
import glob
import json
import os

idx = 0
outer_idx = 0
file = open('listfile-val.lst', 'w') 

for filename in glob.glob('data/flw-responses/*.json'):
    outer_idx = outer_idx + 1
    data = json.load(open(filename))
    
    if outer_idx < 11414: 
        continue
        
    filename = filename.split('/')[2]
    filename = filename.split('.')[0]
    fname = "data/flw-resize/{}.jpg".format(filename)
    
    if os.path.isfile(fname):
        idx = idx + 1
        print('.', end='')
        smile = 0
        if data['Smile']['Value'] and data['Smile']['Confidence']> 0.75:
            smile = 1
            
        emotions = ['HAPPY','CALM','ANGRY','SAD','SURPRISED','CONFUSED','DISGUSTED']
        labels = [0, 0, 0, 0, 0, 0, 0]
        
        i=-1
        for emotion in emotions:
            i = i+1
            for data_emotion in data['Emotions']:
                if emotion == data_emotion['Type']: 
                    if data_emotion['Confidence'] > 0.75:
                        labels[i] = 1
        
        file.write("{} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {:-d} \t {} \n".format(
            idx, smile, labels[0], labels[1], labels[2], labels[3], labels[4], labels[5], labels[6], fname))
    

....................................................................................................................................................................................................................................................................................................................................................................................

.....................................................................................................................................................................................................................................................................................................................................................................................................................

.......................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................

............................................................................................................